In [ ]:
# from google.colab import drive
# drive.mount("/content/drive")

In [ ]:
import math
import torchtext
import torch
import torch.nn as nn
from torchtext.vocab import vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torch import Tensor
from torch.nn import (TransformerEncoder, TransformerDecoder,TransformerEncoderLayer, TransformerDecoderLayer)
import io
import time
from collections import OrderedDict
import csv

In [ ]:
##### drive path #####
# BPE_FILEPATH = "drive/MyDrive/dissertation_workplace/code_translation/preprocessed_files/BPE"
# OUTPUT_FILEPATH = "drive/MyDrive/dissertation_workplace/code_translation/output_files"

##### kaggle path #####
BPE_FILEPATH = "../input/code-translation-v4/preprocessed_files/BPE"
OUTPUT_FILEPATH = "./"

SRC_LANGUAGE = "pn"
TGT_LANGUAGE = "ja"
MAX_COUNT = 10000

# Place-holders
token_transform = {}
vocab_transform = {}

In [ ]:
def build_vocab(filename):
  ordered_dict = OrderedDict()
  with io.open(filename) as file:
    for string_ in file:
      word_feq = string_.rstrip("\n").split(" ")
      word = word_feq[0]
      feq = int(word_feq[1])
      ordered_dict[word] = feq
  vocabulary = vocab(ordered_dict)
  unk_token = "<unk>"
  if unk_token not in vocabulary: vocabulary.insert_token(unk_token, 0)
  vocabulary.insert_token("<pad>", 1)
  vocabulary.insert_token("<bos>", 2)
  vocabulary.insert_token("<eos>", 3)
  vocabulary.set_default_index(vocabulary[unk_token])
  return vocabulary

In [ ]:
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  vocab_transform[ln] = build_vocab(f"{BPE_FILEPATH}/vocab.{ln}.{MAX_COUNT}")

In [ ]:
def data_process(filepaths):
  raw_src_iter = iter(io.open(filepaths[0]))
  raw_tgt_iter = iter(io.open(filepaths[1]))
  data = []
  for (raw_src, raw_tgt) in zip(raw_src_iter, raw_tgt_iter):
    src_tensor_ = torch.tensor([vocab_transform[SRC_LANGUAGE][token] for token in raw_src.rstrip("\n").split(" ")], dtype=torch.long)
    tgt_tensor_ = torch.tensor([vocab_transform[TGT_LANGUAGE][token] for token in raw_tgt.rstrip("\n").split(" ")], dtype=torch.long)
    data.append((src_tensor_, tgt_tensor_))
  return data

In [ ]:
train_data = data_process([f"{BPE_FILEPATH}/train.{SRC_LANGUAGE}.{MAX_COUNT}", f"{BPE_FILEPATH}/train.{TGT_LANGUAGE}.{MAX_COUNT}"])

In [ ]:
BATCH_SIZE = 4
PAD_IDX = vocab_transform[SRC_LANGUAGE]["<pad>"]
BOS_IDX = vocab_transform[SRC_LANGUAGE]["<bos>"]
EOS_IDX = vocab_transform[SRC_LANGUAGE]["<eos>"]

In [ ]:
# DataLoader
def generate_batch(data_batch):
  src_batch, tgt_batch = [], []
  for (src_item, tgt_item) in data_batch:
    src_batch.append(torch.cat([torch.tensor([BOS_IDX]), src_item, torch.tensor([EOS_IDX])], dim=0))
    tgt_batch.append(torch.cat([torch.tensor([BOS_IDX]), tgt_item, torch.tensor([EOS_IDX])], dim=0))
  src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
  tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
  return src_batch, tgt_batch

In [ ]:
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)

In [ ]:
# transformer
class Seq2SeqTransformer(nn.Module):
    def __init__(self, 
                 num_encoder_layers: int, 
                 num_decoder_layers: int,
                 emb_size: int, 
                 nhead:int,
                 src_vocab_size: int, 
                 tgt_vocab_size: int,
                 dim_feedforward:int, 
                 activation:str,
                 layer_norm_eps:float,
                 dropout:float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        encoder_layer = TransformerEncoderLayer(d_model = emb_size, 
                                                nhead = nhead,
                                                dim_feedforward = dim_feedforward, 
                                                activation = activation, 
                                                layer_norm_eps = layer_norm_eps)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers = num_encoder_layers)

        decoder_layer = TransformerDecoderLayer(d_model = emb_size, 
                                                nhead = nhead, 
                                                dim_feedforward = dim_feedforward,
                                                activation = activation, 
                                                layer_norm_eps = layer_norm_eps)
        self.transformer_decoder = TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)

        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(emb_size, dropout = dropout)

    def forward(self, src: Tensor, trg: Tensor, src_mask: Tensor,
                tgt_mask: Tensor, src_padding_mask: Tensor,
                tgt_padding_mask: Tensor, memory_key_padding_mask: Tensor):
      
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        memory = self.transformer_encoder(src_emb, src_mask, src_padding_mask)
        outs = self.transformer_decoder(tgt_emb, memory, tgt_mask, None, tgt_padding_mask, memory_key_padding_mask)

        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer_encoder(self.positional_encoding(self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer_decoder(self.positional_encoding(self.tgt_tok_emb(tgt)), memory, tgt_mask)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, 
                 emb_size: int, 
                 dropout, 
                 maxlen: int = 450):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2) * math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer("pos_embedding", pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding +
                            self.pos_embedding[:token_embedding.size(0),:])

class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size
        
    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

In [ ]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=DEVICE)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float("-inf")).masked_fill(mask == 1, float(0.0))
    return mask

In [ ]:
def create_mask(src, tgt):
    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len)
    src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool)

    src_padding_mask = (src == PAD_IDX).transpose(0, 1)
    tgt_padding_mask = (tgt == PAD_IDX).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512 # default = 512 in Pytorch
NHEAD = 8 # default = 8 in Pytorch
FFN_HID_DIM = 512 # default = 2048 in Pytorch
# BATCH_SIZE = 128 # already defined above
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3
NUM_EPOCHS = 50
LEARNING_RATE = 5e-5 # default = 1e-3 in Pytorch
ACTIVATION = "relu" # default = "relu" in Pytorch
LAYER_NORM_EPS = 1e-5 # default = 1e-5 in Pytorch
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
OUTPUT_MODEL = f"{OUTPUT_FILEPATH}/sourcecode_nmt_{SRC_LANGUAGE}2{TGT_LANGUAGE}_{MAX_COUNT}C_{NUM_EPOCHS}E_{LEARNING_RATE}LR_{ACTIVATION}.pth"

In [ ]:
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE, NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM, ACTIVATION, LAYER_NORM_EPS)

for p in transformer.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)
# default values
optimizer = torch.optim.Adam(transformer.parameters(), lr=LEARNING_RATE, betas=(0.9, 0.999), eps=1e-8)

In [ ]:
def train_epoch(model, train_iter, optimizer):
    model.train()
    losses = 0
    for idx, (src, tgt) in enumerate(train_iter):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input)

        logits = model(src, tgt_input, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, src_padding_mask)

        optimizer.zero_grad()
        
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        
        optimizer.step()
        losses += loss.item()
    return losses / len(train_iter)


In [ ]:
from timeit import default_timer as timer

loss_data = []
start_training_time = timer()
for epoch in range(NUM_EPOCHS):
    start_time = timer()
    train_loss = train_epoch(transformer, train_iter, optimizer)
    end_time = timer()
    loss_data.append({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "epoch_time": (end_time - start_time)
    })
    print((f"Epoch: {epoch + 1}, Train loss: {train_loss:.3f} Epoch time = {(end_time - start_time):.3f}s"))
end_training_time = timer()

In [ ]:
torch.save(transformer, OUTPUT_MODEL)

In [ ]:
time_taken_to_train = end_training_time - start_training_time
mins = time_taken_to_train/60 
if mins < 60:
    train_time = f"Training time: {mins} mins"
    print(f"Training time taken: {mins} mins")
else:
    hrs = int(mins/60)
    mins = mins - hrs * 60
    train_time = f"Training time taken: {hrs} hrs {mins} mins"
    print(f"Training time taken: {hrs} hrs {mins} mins")

In [ ]:
# write loss to csv file
field_names = ["epoch", "train_loss", "epoch_time"]
CSV_FILE = f"{OUTPUT_MODEL[:-4]}.csv"
with open(CSV_FILE, "w") as outfile:
  writer = csv.DictWriter(outfile, fieldnames = field_names)
  writer.writeheader()
  writer.writerows(loss_data)

In [ ]:
# write time taken to file
TIME_FILE = f"{OUTPUT_MODEL[:-4]}.txt"
with open(TIME_FILE, "w") as outfile:
    outfile.write(train_time)